# Optimizing Donation Allocation for the Fondation Pierre et Yolande Gingras
### MGSC 662 Final Project
- Sofia Berumen Ramos - 261257231
- Romane Lucas-Girardville - 261259042
- Andrea Vreugdenhil - 260991131
- Gina Yassin - 261261130


In [2]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [24]:
# ============================================================================
# BASE MODEL (Weighted Sum)
# ============================================================================

# Create a new clean model
m1 = Model("Base Weighted Sum Model: Donation Allocation Optimization")

# ============================================================================
# Load and Prepare Data
# ============================================================================

data = pd.read_excel('project_data.xlsx', sheet_name='Data')
weight = pd.read_excel('project_data.xlsx', sheet_name='Weights')

# Extract project data
P = data['project_id'].tolist()
charity = dict(zip(data['project_id'], data['charity']))
description = dict(zip(data['project_id'], data['description']))
term = dict(zip(data['project_id'], data['term']))
cost_cap_1_it = dict(zip(data['project_id'], data['cost_cap_1_it']))
requires_full = dict(zip(data['project_id'], data['requires_full']))
min_amount = dict(zip(data['project_id'], data['min_amount']))
depends_on = dict(zip(data['project_id'], data['depends_on']))
max_no = dict(zip(data['project_id'], data['max_no']))
edu = dict(zip(data['project_id'], data['edu']))
comm = dict(zip(data['project_id'], data['comm']))
health = dict(zip(data['project_id'], data['health']))
total_weight = dict(zip(data['project_id'], data['total_weight']))

# Separate projects by charity
JMDM_p = [p for p in P if charity[p] == 'JMDM']
CSM_p = [p for p in P if charity[p] == 'CSM']

# Separate projects by term
LongTerm_p = [p for p in P if term[p] == 'LongTerm']

# Dependency dictionary
dependencies = {p: int(depends_on[p]) for p in P if pd.notna(depends_on[p])}

# ============================================================================
# Parameters
# ============================================================================

B = 50000   # Total budget
AJ = 10000  # Minimum allocation to JMDM
AC = 15000  # Minimum allocation to CSM
L = 0.75    # Minimum 75% to long-term projects

# Charity-specific impact weights
charity_weights = {
    'JMDM': {
        'edu_weight': weight.loc[weight['charity'] == 'JMDM', 'edu'].values[0],
        'comm_weight': weight.loc[weight['charity'] == 'JMDM', 'comm'].values[0],
        'health_weight': weight.loc[weight['charity'] == 'JMDM', 'health'].values[0],
    },
    'CSM': {
        'edu_weight': weight.loc[weight['charity'] == 'CSM', 'edu'].values[0],
        'comm_weight': weight.loc[weight['charity'] == 'CSM', 'comm'].values[0],
        'health_weight': weight.loc[weight['charity'] == 'CSM', 'health'].values[0],
    }
}

# ============================================================================
# Decision Variables
# ============================================================================

# Amount allocated to project p 
x = m1.addVars(P, vtype=GRB.CONTINUOUS, name="x", lb=0)

# Number of iterations for project p 
y = m1.addVars(P, vtype=GRB.INTEGER, name="y", lb=0)


# ============================================================================
# Objective Function
# ============================================================================
# Maximize weighted impact based on funding ratio

m1.setObjective(gp.quicksum((x[p] / cost_cap_1_it[p]) * total_weight[p] for p in P), GRB.MAXIMIZE)

# ============================================================================
# Constraints
# ============================================================================

# Budget Constraint
m1.addConstr(gp.quicksum(x[p] for p in P) <= B, name="Budget")

# Long-term Allocation Minimum 
m1.addConstr(gp.quicksum(x[p] for p in LongTerm_p) >= L * gp.quicksum(x[p] for p in P), name="LongTerm_Minimum")

# Charity Minimum Allocations
m1.addConstr(gp.quicksum(x[p] for p in JMDM_p) >= AJ, name="JMDM_Minimum")
m1.addConstr(gp.quicksum(x[p] for p in CSM_p) >= AC, name="CSM_Minimum")

# Project-Level Constraints
for p in P:

    # Total allocation cannot exceed cost cap x iterations
    m1.addConstr(x[p] <= cost_cap_1_it[p] * y[p], name=f"CostCap_{p}")
    
    # Iterations cannot exceed maximum allowed
    m1.addConstr(y[p] <= max_no[p], name=f"MaxIterations_{p}")
    
    # Minimum funding if project is funded
    m1.addConstr(x[p] >= min_amount[p] * y[p], name=f"MinFunding_{p}")


# Dependent project can only have iterations if prerequisite has at least 1 iteration
for p, dep in dependencies.items():
    m1.addConstr(y[p] <= max_no[p] * y[dep], name=f"Dependency_{dep}_to_{p}")


# ============================================================================
# Optimize
# ============================================================================

m1.optimize()

# ============================================================================
# Display Results
# ============================================================================

# AI was used to made the print more readable

print("\n" + "="*80)
print("BASE MODEL - OPTIMAL SOLUTION")
print("="*80)

# Calculate totals
total_jmdm = sum(x[p].x for p in JMDM_p if x[p].x > 0)
total_csm = sum(x[p].x for p in CSM_p if x[p].x > 0)
total_allocated = sum(x[p].x for p in P)
total_longterm = sum(x[p].x for p in LongTerm_p if x[p].x > 0)

print(f"\nBUDGET ALLOCATION:")
print(f"   Total Budget: ${B:,.2f}")
print(f"   Total Allocated: ${total_allocated:,.2f} ({total_allocated/B*100:.1f}%)")
print(f"   Remaining: ${B - total_allocated:,.2f}")

print(f"\nCHARITY BREAKDOWN:")
print(f"   JMDM: ${total_jmdm:,.2f} ({total_jmdm/total_allocated*100:.1f}%)")
print(f"   CSM:  ${total_csm:,.2f} ({total_csm/total_allocated*100:.1f}%)")

print(f"\nPROJECT TERM BREAKDOWN:")
print(f"   Long-term: ${total_longterm:,.2f} ({total_longterm/total_allocated*100:.1f}%)")
print(f"   Ad-hoc: ${total_allocated - total_longterm:,.2f} ({(total_allocated - total_longterm)/total_allocated*100:.1f}%)")

print(f"\nFUNDED PROJECTS ({sum(1 for p in P if x[p].x > 0)} out of {len(P)}):")
print(f"{'ID':<5} {'Charity':<8} {'Description':<40} {'Amount':<12} {'Iter':<6} {'Term':<10}")
print("-" * 85)

for p in P:
    if x[p].x > 0:
        print(f"{p:<5} {charity[p]:<8} {description[p][:38]:<40} ${x[p].x:>10,.2f} {round(y[p].x):<6} {term[p]:<10}")

print("\n" + "="*80)
print(f"Objective Value (Total Weighted Impact): {m1.objVal:.2f}")
print("="*80)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 7530U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 71 rows, 40 columns and 174 nonzeros
Model fingerprint: 0x2f7a0d82
Variable types: 20 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+04]
  Objective range  [9e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 50 rows and 17 columns
Presolve time: 0.00s
Presolved: 21 rows, 23 columns, 87 nonzeros
Variable types: 5 continuous, 18 integer (11 binary)
Found heuristic solution: objective 76.5866667
Found heuristic solution: objective 88.7333333
Found heuristic solution: objective 90.0666667

Root relaxation: objective 1.080800e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

In [22]:
# ============================================================================
# GOAL PROGRAMMING MODEL 
# ============================================================================

# Create goal programming model
m2 = Model("Goal Programming Model: Donation Allocation Optimization")
m2.setParam('Seed', 123)
m2.setParam('Method', 1)
m2.setParam('Threads', 1)
m2.setParam('MIPGap', 0)
m2.setParam('NodeMethod', 1)

# ============================================================================
# Load and Prepare Data (same as base model)
# ============================================================================

data = pd.read_excel('project_data.xlsx', sheet_name='Data')
weight = pd.read_excel('project_data.xlsx', sheet_name='Weights')

# Extract project data
P = data['project_id'].tolist()
charity = dict(zip(data['project_id'], data['charity']))
description = dict(zip(data['project_id'], data['description']))
term = dict(zip(data['project_id'], data['term']))
cost_cap_1_it = dict(zip(data['project_id'], data['cost_cap_1_it']))
requires_full = dict(zip(data['project_id'], data['requires_full']))
min_amount = dict(zip(data['project_id'], data['min_amount']))
depends_on = dict(zip(data['project_id'], data['depends_on']))
max_no = dict(zip(data['project_id'], data['max_no']))
edu = dict(zip(data['project_id'], data['edu']))
comm = dict(zip(data['project_id'], data['comm']))
health = dict(zip(data['project_id'], data['health']))
total_weight = dict(zip(data['project_id'], data['total_weight']))

# Separate projects by charity
JMDM_p = [p for p in P if charity[p] == 'JMDM']
CSM_p = [p for p in P if charity[p] == 'CSM']

# Separate projects by term
LongTerm_p = [p for p in P if term[p] == 'LongTerm']

# Dependency dictionary
dependencies = {p: int(depends_on[p]) for p in P if pd.notna(depends_on[p])}

# ============================================================================
# Parameters
# ============================================================================

B = 50000   # Total budget
AJ = 10000  # Minimum allocation to JMDM
AC = 15000  # Minimum allocation to CSM
L = 0.75    # Minimum 75% to long-term projects

# Charity-specific impact weights 
charity_weights = {
    'JMDM': {
        'edu_weight': weight.loc[weight['charity'] == 'JMDM', 'edu'].values[0],
        'comm_weight': weight.loc[weight['charity'] == 'JMDM', 'comm'].values[0],
        'health_weight': weight.loc[weight['charity'] == 'JMDM', 'health'].values[0],
    },
    'CSM': {
        'edu_weight': weight.loc[weight['charity'] == 'CSM', 'edu'].values[0],
        'comm_weight': weight.loc[weight['charity'] == 'CSM', 'comm'].values[0],
        'health_weight': weight.loc[weight['charity'] == 'CSM', 'health'].values[0],
    }
}

# ============================================================================
# Goal Programming Parameters
# ============================================================================

# Target proportions 
# JMDM: Education = 100%, Health = 80%, Community = 40%
# CSM: Community = 100%, Education = 80%, Health = 40%

JMDM_proportions = {
    'edu': 100 / (100 + 80 + 40),      # 45.45%
    'health': 80 / (100 + 80 + 40),    # 36.36%
    'comm': 40 / (100 + 80 + 40)       # 18.18%
}

CSM_proportions = {
    'comm': 100 / (100 + 80 + 40),     # 45.45%
    'edu': 80 / (100 + 80 + 40),       # 36.36%
    'health': 40 / (100 + 80 + 40)     # 18.18%
}

W_PROPORTION = 5.0   # Weight for proportional deviations (BALANCED scenario)
W_BUDGET = 100.0      # Weight for budget utilization
W_TOTAL_IMPACT = 1.0 # Weight for total impact maximization (BALANCED scenario)

# ============================================================================
# Decision Variables
# ============================================================================

# Primary decision variables
x = m2.addVars(P, vtype=GRB.CONTINUOUS, name="x", lb=0)
y = m2.addVars(P, vtype=GRB.INTEGER, name="y", lb=0)

# Auxiliary variables for CALIBRATED scores (RAW scores without weights)
# These represent: (x_i / C_i) * raw_score
JMDM_edu_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="JMDM_edu_calibrated", lb=0)
JMDM_health_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="JMDM_health_calibrated", lb=0)
JMDM_comm_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="JMDM_comm_calibrated", lb=0)
JMDM_total_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="JMDM_total_calibrated", lb=0)

CSM_edu_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="CSM_edu_calibrated", lb=0)
CSM_health_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="CSM_health_calibrated", lb=0)
CSM_comm_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="CSM_comm_calibrated", lb=0)
CSM_total_calibrated = m2.addVar(vtype=GRB.CONTINUOUS, name="CSM_total_calibrated", lb=0)

# Deviation variables for proportional goals
d_JMDM_edu_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_edu_neg", lb=0)
d_JMDM_edu_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_edu_pos", lb=0)

d_JMDM_health_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_health_neg", lb=0)
d_JMDM_health_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_health_pos", lb=0)

d_JMDM_comm_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_comm_neg", lb=0)
d_JMDM_comm_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_comm_pos", lb=0)

d_CSM_edu_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_edu_neg", lb=0)
d_CSM_edu_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_edu_pos", lb=0)

d_CSM_health_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_health_neg", lb=0)
d_CSM_health_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_health_pos", lb=0)

d_CSM_comm_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_comm_neg", lb=0)
d_CSM_comm_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_comm_pos", lb=0)

# Budget utilization deviation variables
d_budget_neg = m2.addVar(vtype=GRB.CONTINUOUS, name="d_budget_neg", lb=0)
d_budget_pos = m2.addVar(vtype=GRB.CONTINUOUS, name="d_budget_pos", lb=0)

# ============================================================================
# Objective Function
# ============================================================================
# Note: Total impact uses WEIGHTED scores for the objective function
# But proportional deviations use UNWEIGHTED (raw) scores

total_impact = gp.quicksum((x[p] / cost_cap_1_it[p]) * total_weight[p] for p in P)

m2.setObjective(
    W_BUDGET * d_budget_neg +
    W_PROPORTION * (
        d_JMDM_edu_neg + d_JMDM_edu_pos +
        d_JMDM_health_neg + d_JMDM_health_pos +
        d_JMDM_comm_neg + d_JMDM_comm_pos +
        d_CSM_edu_neg + d_CSM_edu_pos +
        d_CSM_health_neg + d_CSM_health_pos +
        d_CSM_comm_neg + d_CSM_comm_pos
    ) - W_TOTAL_IMPACT * total_impact,
    GRB.MINIMIZE
)

# ============================================================================
# Calibrated Score Constraints
# ============================================================================

# JMDM calibrated scores (sum of (x_i/C_i) * raw_score for each category)
m2.addConstr(JMDM_edu_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * edu[p] for p in JMDM_p),    name="JMDM_edu_calibrated")
m2.addConstr(JMDM_health_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * health[p] for p in JMDM_p),    name="JMDM_health_calibrated")
m2.addConstr(JMDM_comm_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * comm[p] for p in JMDM_p),    name="JMDM_comm_calibrated")

# CSM calibrated scores
m2.addConstr(CSM_edu_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * edu[p] for p in CSM_p),    name="CSM_edu_calibrated")
m2.addConstr(CSM_health_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * health[p] for p in CSM_p),    name="CSM_health_calibrated")
m2.addConstr(CSM_comm_calibrated == gp.quicksum((x[p] / cost_cap_1_it[p]) * comm[p] for p in CSM_p),    name="CSM_comm_calibrated")

# Total calibrated scores for each charity (sum across all categories)
m2.addConstr(JMDM_total_calibrated == JMDM_edu_calibrated + JMDM_health_calibrated + JMDM_comm_calibrated,    name="JMDM_total_calibrated")
m2.addConstr(CSM_total_calibrated == CSM_edu_calibrated + CSM_health_calibrated + CSM_comm_calibrated,    name="CSM_total_calibrated")

# ============================================================================
# Proportional Goal Constraints 
# ============================================================================
# JMDM Proportional Goals
m2.addConstr(JMDM_edu_calibrated + d_JMDM_edu_neg - d_JMDM_edu_pos == 
    JMDM_proportions['edu'] * JMDM_total_calibrated,
    name="JMDM_edu_proportion")

m2.addConstr(JMDM_health_calibrated + d_JMDM_health_neg - d_JMDM_health_pos == 
    JMDM_proportions['health'] * JMDM_total_calibrated,
    name="JMDM_health_proportion")

m2.addConstr(JMDM_comm_calibrated + d_JMDM_comm_neg - d_JMDM_comm_pos == 
    JMDM_proportions['comm'] * JMDM_total_calibrated,
    name="JMDM_comm_proportion")

# CSM Proportional Goals
m2.addConstr(CSM_edu_calibrated + d_CSM_edu_neg - d_CSM_edu_pos == 
    CSM_proportions['edu'] * CSM_total_calibrated,
    name="CSM_edu_proportion")

m2.addConstr(CSM_health_calibrated + d_CSM_health_neg - d_CSM_health_pos == 
    CSM_proportions['health'] * CSM_total_calibrated,
    name="CSM_health_proportion")

m2.addConstr(CSM_comm_calibrated + d_CSM_comm_neg - d_CSM_comm_pos == 
    CSM_proportions['comm'] * CSM_total_calibrated,
    name="CSM_comm_proportion")

# Budget Utilization Goal
m2.addConstr(gp.quicksum(x[p] for p in P) + d_budget_neg - d_budget_pos == B, name="Budget_Utilization_Goal")

# ============================================================================
# Hard Constraints (same as base model)
# ============================================================================

# Budget Constraint
m2.addConstr(gp.quicksum(x[p] for p in P) <= B, name="Budget")

# Long-term Allocation Minimum
m2.addConstr(gp.quicksum(x[p] for p in LongTerm_p) >= L * gp.quicksum(x[p] for p in P),name="LongTerm_Minimum")

# Charity Minimum Allocations
m2.addConstr(gp.quicksum(x[p] for p in JMDM_p) >= AJ, name="JMDM_Minimum")
m2.addConstr(gp.quicksum(x[p] for p in CSM_p) >= AC, name="CSM_Minimum")

# Dependency Constraints
for p, dep in dependencies.items():
    m2.addConstr(y[p] <= max_no[p] * y[dep], name=f"Dependency_{dep}_to_{p}")

# Project-Level Constraints
for p in P:
    m2.addConstr(x[p] <= cost_cap_1_it[p] * y[p], name=f"CostCap_{p}")
    m2.addConstr(y[p] <= max_no[p], name=f"MaxIterations_{p}")
    m2.addConstr(x[p] >= min_amount[p] * y[p], name=f"MinFunding_{p}")

# ============================================================================
# Optimize
# ============================================================================

m2.optimize()

# ============================================================================
# Display Results
# ============================================================================

# AI was used to made the output more readable

if m2.status == GRB.OPTIMAL:
    print("\n" + "="*80)
    print("GOAL PROGRAMMING MODEL - OPTIMAL SOLUTION (CORRECTED)")
    print("="*80)

    # Calculate actual calibrated scores
    actual_JMDM_edu = JMDM_edu_calibrated.x
    actual_JMDM_health = JMDM_health_calibrated.x
    actual_JMDM_comm = JMDM_comm_calibrated.x
    actual_JMDM_total = JMDM_total_calibrated.x

    actual_CSM_edu = CSM_edu_calibrated.x
    actual_CSM_health = CSM_health_calibrated.x
    actual_CSM_comm = CSM_comm_calibrated.x
    actual_CSM_total = CSM_total_calibrated.x

    print("\nJMDM PROPORTIONAL ACHIEVEMENT (Based on RAW Scores):")
    print(f"{'Category':<15} {'Target %':<12} {'Calibrated':<15} {'Actual %':<12} {'Deviation':<12}")
    print("-" * 70)
    if actual_JMDM_total > 0:
        print(f"{'Education':<15} {JMDM_proportions['edu']*100:>10.1f}% "
              f"{actual_JMDM_edu:>14.2f} "
              f"{(actual_JMDM_edu/actual_JMDM_total)*100:>10.1f}% "
              f"{d_JMDM_edu_neg.x + d_JMDM_edu_pos.x:>11.2f}")
        print(f"{'Health':<15} {JMDM_proportions['health']*100:>10.1f}% "
              f"{actual_JMDM_health:>14.2f} "
              f"{(actual_JMDM_health/actual_JMDM_total)*100:>10.1f}% "
              f"{d_JMDM_health_neg.x + d_JMDM_health_pos.x:>11.2f}")
        print(f"{'Community':<15} {JMDM_proportions['comm']*100:>10.1f}% "
              f"{actual_JMDM_comm:>14.2f} "
              f"{(actual_JMDM_comm/actual_JMDM_total)*100:>10.1f}% "
              f"{d_JMDM_comm_neg.x + d_JMDM_comm_pos.x:>11.2f}")
    print(f"{'TOTAL':<15} {'100.0%':>10} {actual_JMDM_total:>14.2f}")

    print("\nCSM PROPORTIONAL ACHIEVEMENT (Based on RAW Scores):")
    print(f"{'Category':<15} {'Target %':<12} {'Calibrated':<15} {'Actual %':<12} {'Deviation':<12}")
    print("-" * 70)
    if actual_CSM_total > 0:
        print(f"{'Community':<15} {CSM_proportions['comm']*100:>10.1f}% "
              f"{actual_CSM_comm:>14.2f} "
              f"{(actual_CSM_comm/actual_CSM_total)*100:>10.1f}% "
              f"{d_CSM_comm_neg.x + d_CSM_comm_pos.x:>11.2f}")
        print(f"{'Education':<15} {CSM_proportions['edu']*100:>10.1f}% "
              f"{actual_CSM_edu:>14.2f} "
              f"{(actual_CSM_edu/actual_CSM_total)*100:>10.1f}% "
              f"{d_CSM_edu_neg.x + d_CSM_edu_pos.x:>11.2f}")
        print(f"{'Health':<15} {CSM_proportions['health']*100:>10.1f}% "
              f"{actual_CSM_health:>14.2f} "
              f"{(actual_CSM_health/actual_CSM_total)*100:>10.1f}% "
              f"{d_CSM_health_neg.x + d_CSM_health_pos.x:>11.2f}")
    print(f"{'TOTAL':<15} {'100.0%':>10} {actual_CSM_total:>14.2f}")

    # Calculate totals
    total_jmdm = sum(x[p].x for p in JMDM_p if x[p].x > 0.01)
    total_csm = sum(x[p].x for p in CSM_p if x[p].x > 0.01)
    total_allocated = sum(x[p].x for p in P if x[p].x > 0.01)
    total_longterm = sum(x[p].x for p in LongTerm_p if x[p].x > 0.01)
    total_weighted_impact = sum((x[p].x / cost_cap_1_it[p]) * total_weight[p] for p in P)

    print(f"\nBUDGET ALLOCATION:")
    print(f"   Total Budget: ${B:,.2f}")
    print(f"   Total Allocated: ${total_allocated:,.2f} ({total_allocated/B*100:.1f}%)")
    print(f"   Remaining: ${B - total_allocated:,.2f}")

    print(f"\nCHARITY BREAKDOWN:")
    print(f"   JMDM: ${total_jmdm:,.2f} ({total_jmdm/total_allocated*100:.1f}%)")
    print(f"   CSM:  ${total_csm:,.2f} ({total_csm/total_allocated*100:.1f}%)")

    print(f"\nPROJECT TERM BREAKDOWN:")
    print(f"   Long-term: ${total_longterm:,.2f} ({total_longterm/total_allocated*100:.1f}%)")
    print(f"   Ad-hoc: ${total_allocated - total_longterm:,.2f} "
          f"({(total_allocated - total_longterm)/total_allocated*100:.1f}%)")

    print(f"\nFUNDED PROJECTS ({sum(1 for p in P if x[p].x > 0.01)} out of {len(P)}):")
    print(f"{'ID':<5} {'Charity':<8} {'Description':<40} {'Amount':<12} {'Iter':<6} {'Term':<10}")
    print("-" * 85)

    for p in P:
        if x[p].x > 0.01:
            y_rounded = int(round(y[p].x))
            print(f"{p:<5} {charity[p]:<8} {description[p][:38]:<40} "
                  f"${x[p].x:>10,.2f} {y_rounded:<6} {term[p]:<10}")

    print("\n" + "="*80)
    total_deviation = sum([
        d_JMDM_edu_neg.x, d_JMDM_edu_pos.x,
        d_JMDM_health_neg.x, d_JMDM_health_pos.x,
        d_JMDM_comm_neg.x, d_JMDM_comm_pos.x,
        d_CSM_edu_neg.x, d_CSM_edu_pos.x,
        d_CSM_health_neg.x, d_CSM_health_pos.x,
        d_CSM_comm_neg.x, d_CSM_comm_pos.x
    ])
    print(f"Total Proportional Deviation: {total_deviation:.4f}")
    print(f"Total Weighted Impact: {total_weighted_impact:.2f}")
    print(f"Objective Value: {m2.objVal:.2f}")
    print("="*80)

else:
    print(f"\nOptimization was not successful. Status code: {m2.status}")

Set parameter Seed to value 123
Set parameter Method to value 1
Set parameter Threads to value 1
Set parameter MIPGap to value 0
Set parameter NodeMethod to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 7530U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 1 threads

Non-default parameters:
MIPGap  0
Method  1
NodeMethod  1
Seed  123
Threads  1

Optimize a model with 86 rows, 62 columns and 294 nonzeros
Model fingerprint: 0x93d6891b
Variable types: 42 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+04]
  Objective range  [9e-04, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 53 rows and 25 columns
Presolve time: 0.00s
Presolved: 33 rows, 37 columns, 175 nonzeros
Variable types: 18 continuous, 19 integer (12 binary)
Found heuristic solution: objective 68.5430303
F

In [25]:
# ============================================================================
# SENSITIVITY ANALYSIS - Weight Variation
# ============================================================================

# Store for reference
BASE_MODEL_IMPACT = m1.objVal

sensitivity_results = []

weight_scenarios = [
    ("Current", 5, 100, 1),
    ("High Proportion", 50, 100, 1),
    ("Very High Proportion", 100, 100, 1),
    ("High Impact", 1, 100, 50),
    ("Favor Proportion", 25, 100, 10),
    ("Balanced", 10, 100, 10),
]

# to ensure t
for scenario_name, w_prop, w_budget, w_impact in weight_scenarios:
    m_sens = Model(f"Sensitivity_{scenario_name}")
    m_sens.setParam('OutputFlag', 0)  # Suppress output
    m_sens.setParam('Seed', 123)  # Set random seed for reproducibility
    m_sens.setParam('Method', 1)  # Use dual simplex method for consistency
    m_sens.setParam('Threads', 1)  # Use single thread for deterministic results
    m_sens.setParam('MIPGap', 0)  # Require exact optimal solution
    m_sens.setParam('NodeMethod', 1)  # Use dual simplex at MIP nodes
    
    # ============================================================================
    # Decision Variables
    # ============================================================================
    
    x_sens = m_sens.addVars(P, vtype=GRB.CONTINUOUS, name="x", lb=0)
    y_sens = m_sens.addVars(P, vtype=GRB.INTEGER, name="y", lb=0)
    
    # Auxiliary variables for CALIBRATED scores (RAW scores without weights)
    JMDM_edu_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="JMDM_edu_calibrated", lb=0)
    JMDM_health_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="JMDM_health_calibrated", lb=0)
    JMDM_comm_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="JMDM_comm_calibrated", lb=0)
    JMDM_total_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="JMDM_total_calibrated", lb=0)
    
    CSM_edu_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="CSM_edu_calibrated", lb=0)
    CSM_health_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="CSM_health_calibrated", lb=0)
    CSM_comm_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="CSM_comm_calibrated", lb=0)
    CSM_total_calibrated_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="CSM_total_calibrated", lb=0)
    
    # Deviation variables for proportional goals
    d_JMDM_edu_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_edu_neg", lb=0)
    d_JMDM_edu_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_edu_pos", lb=0)
    d_JMDM_health_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_health_neg", lb=0)
    d_JMDM_health_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_health_pos", lb=0)
    d_JMDM_comm_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_comm_neg", lb=0)
    d_JMDM_comm_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_JMDM_comm_pos", lb=0)
    
    d_CSM_edu_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_edu_neg", lb=0)
    d_CSM_edu_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_edu_pos", lb=0)
    d_CSM_health_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_health_neg", lb=0)
    d_CSM_health_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_health_pos", lb=0)
    d_CSM_comm_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_comm_neg", lb=0)
    d_CSM_comm_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_CSM_comm_pos", lb=0)
    
    d_budget_neg_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_budget_neg", lb=0)
    d_budget_pos_sens = m_sens.addVar(vtype=GRB.CONTINUOUS, name="d_budget_pos", lb=0)
    
    # ============================================================================
    # Calculate total impact for objective
    # ============================================================================
    
    total_impact_sens = gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * total_weight[p] for p in P)
    
    # ============================================================================
    # Objective Function with scenario-specific weights
    # ============================================================================
    
    m_sens.setObjective(
        w_budget * d_budget_neg_sens +
        w_prop * (
            d_JMDM_edu_neg_sens + d_JMDM_edu_pos_sens +
            d_JMDM_health_neg_sens + d_JMDM_health_pos_sens +
            d_JMDM_comm_neg_sens + d_JMDM_comm_pos_sens +
            d_CSM_edu_neg_sens + d_CSM_edu_pos_sens +
            d_CSM_health_neg_sens + d_CSM_health_pos_sens +
            d_CSM_comm_neg_sens + d_CSM_comm_pos_sens
        ) - w_impact * total_impact_sens,
        GRB.MINIMIZE
    )
    
    # ============================================================================
    # Calibrated Score Constraints
    # ============================================================================
    
    # JMDM calibrated scores
    m_sens.addConstr(JMDM_edu_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * edu[p] for p in JMDM_p),name="JMDM_edu_calibrated")
    m_sens.addConstr(JMDM_health_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * health[p] for p in JMDM_p), name="JMDM_health_calibrated" )
    m_sens.addConstr(JMDM_comm_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * comm[p] for p in JMDM_p), name="JMDM_comm_calibrated"    )
    
    # CSM calibrated scores
    m_sens.addConstr(CSM_edu_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * edu[p] for p in CSM_p),name="CSM_edu_calibrated"   )
    m_sens.addConstr(CSM_health_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * health[p] for p in CSM_p),name="CSM_health_calibrated"    )
    m_sens.addConstr(CSM_comm_calibrated_sens == gp.quicksum((x_sens[p] / cost_cap_1_it[p]) * comm[p] for p in CSM_p),name="CSM_comm_calibrated"    )
    
    # Total calibrated scores for each charity
    m_sens.addConstr(JMDM_total_calibrated_sens == JMDM_edu_calibrated_sens + JMDM_health_calibrated_sens + JMDM_comm_calibrated_sens,name="JMDM_total_calibrated"  )
    m_sens.addConstr(CSM_total_calibrated_sens == CSM_edu_calibrated_sens + CSM_health_calibrated_sens + CSM_comm_calibrated_sens,name="CSM_total_calibrated" )
    
    # ============================================================================
    # Proportional Goal Constraints (using calibrated scores)
    # ============================================================================
    
    # JMDM Proportional Goals
    m_sens.addConstr(JMDM_edu_calibrated_sens + d_JMDM_edu_neg_sens - d_JMDM_edu_pos_sens == 
        JMDM_proportions['edu'] * JMDM_total_calibrated_sens,
        name="JMDM_edu_proportion")
    
    m_sens.addConstr(JMDM_health_calibrated_sens + d_JMDM_health_neg_sens - d_JMDM_health_pos_sens == 
        JMDM_proportions['health'] * JMDM_total_calibrated_sens,
        name="JMDM_health_proportion")
    
    m_sens.addConstr(JMDM_comm_calibrated_sens + d_JMDM_comm_neg_sens - d_JMDM_comm_pos_sens == 
        JMDM_proportions['comm'] * JMDM_total_calibrated_sens,
        name="JMDM_comm_proportion")
    
    # CSM Proportional Goals
    m_sens.addConstr(CSM_edu_calibrated_sens + d_CSM_edu_neg_sens - d_CSM_edu_pos_sens == 
        CSM_proportions['edu'] * CSM_total_calibrated_sens,
        name="CSM_edu_proportion")
    
    m_sens.addConstr(CSM_health_calibrated_sens + d_CSM_health_neg_sens - d_CSM_health_pos_sens == 
        CSM_proportions['health'] * CSM_total_calibrated_sens,
        name="CSM_health_proportion")
    
    m_sens.addConstr(CSM_comm_calibrated_sens + d_CSM_comm_neg_sens - d_CSM_comm_pos_sens == 
        CSM_proportions['comm'] * CSM_total_calibrated_sens,
        name="CSM_comm_proportion")
    
    # Budget Utilization Goal
    m_sens.addConstr(gp.quicksum(x_sens[p] for p in P) + d_budget_neg_sens - d_budget_pos_sens == B, name="Budget_Utilization_Goal")
    
    # ============================================================================
    # Hard Constraints
    # ============================================================================
    
    # Budget Constraint
    m_sens.addConstr(gp.quicksum(x_sens[p] for p in P) <= B, name="Budget")
    
    # Long-term Allocation Minimum
    m_sens.addConstr(gp.quicksum(x_sens[p] for p in LongTerm_p) >= L * gp.quicksum(x_sens[p] for p in P), name="LongTerm_Minimum")
    
    # Charity Minimum Allocations
    m_sens.addConstr(gp.quicksum(x_sens[p] for p in JMDM_p) >= AJ, name="JMDM_Minimum")
    m_sens.addConstr(gp.quicksum(x_sens[p] for p in CSM_p) >= AC, name="CSM_Minimum")
    
    # Dependency Constraints
    for p, dep in dependencies.items():
        m_sens.addConstr(y_sens[p] <= max_no[p] * y_sens[dep], name=f"Dependency_{dep}_to_{p}")
    
    # Project-Level Constraints
    for p in P:
        m_sens.addConstr(x_sens[p] <= cost_cap_1_it[p] * y_sens[p], name=f"CostCap_{p}")
        m_sens.addConstr(y_sens[p] <= max_no[p], name=f"MaxIterations_{p}")
        m_sens.addConstr(x_sens[p] >= min_amount[p] * y_sens[p], name=f"MinFunding_{p}")
    
    # ============================================================================
    # Optimize
    # ============================================================================
    
    m_sens.optimize()
    
    # ============================================================================
    # Store Results
    # ============================================================================
    

    if m_sens.status == GRB.OPTIMAL:
        total_impact_val = sum((x_sens[p].x / cost_cap_1_it[p]) * total_weight[p] for p in P)
        total_deviation = sum([
            d_JMDM_edu_neg_sens.x, d_JMDM_edu_pos_sens.x,
            d_JMDM_health_neg_sens.x, d_JMDM_health_pos_sens.x,
            d_JMDM_comm_neg_sens.x, d_JMDM_comm_pos_sens.x,
            d_CSM_edu_neg_sens.x, d_CSM_edu_pos_sens.x,
            d_CSM_health_neg_sens.x, d_CSM_health_pos_sens.x,
            d_CSM_comm_neg_sens.x, d_CSM_comm_pos_sens.x
        ])
        
        # JMDM calibrated values
        jmdm_edu = JMDM_edu_calibrated_sens.x
        jmdm_health = JMDM_health_calibrated_sens.x
        jmdm_comm = JMDM_comm_calibrated_sens.x
        jmdm_total = JMDM_total_calibrated_sens.x
        
        # CSM calibrated values
        csm_edu = CSM_edu_calibrated_sens.x
        csm_health = CSM_health_calibrated_sens.x
        csm_comm = CSM_comm_calibrated_sens.x
        csm_total = CSM_total_calibrated_sens.x
        
        # Budget utilization
        budget_used = sum(x_sens[p].x for p in P)
        
        sensitivity_results.append({
            'scenario': scenario_name,
            'w_prop': w_prop,
            'w_budget': w_budget,
            'w_impact': w_impact,
            'total_impact': total_impact_val,
            'total_deviation': total_deviation,
            'budget_used': budget_used,
            'budget_pct': budget_used / B * 100,
            'jmdm_edu_pct': jmdm_edu / jmdm_total * 100 if jmdm_total > 0 else 0,
            'jmdm_health_pct': jmdm_health / jmdm_total * 100 if jmdm_total > 0 else 0,
            'jmdm_comm_pct': jmdm_comm / jmdm_total * 100 if jmdm_total > 0 else 0,
            'csm_comm_pct': csm_comm / csm_total * 100 if csm_total > 0 else 0,
            'csm_edu_pct': csm_edu / csm_total * 100 if csm_total > 0 else 0,
            'csm_health_pct': csm_health / csm_total * 100 if csm_total > 0 else 0,
        })
        
        # Store project allocations for this scenario
        sensitivity_results[-1]['projects'] = {p: {'amount': x_sens[p].x, 'iterations': round(y_sens[p].x)} for p in P if x_sens[p].x > 0.01}
        
        # Debug output
        print(f"\n{scenario_name} (w_prop={w_prop}, w_impact={w_impact}):")
        print(f"  Total Impact: {total_impact_val:.2f}")
        print(f"  Total Deviation: {total_deviation:.4f}")
        if jmdm_total > 0:
            print(f"  JMDM - Edu: {jmdm_edu/jmdm_total*100:.1f}% (target 45.5%), Health: {jmdm_health/jmdm_total*100:.1f}% (target 36.4%), Comm: {jmdm_comm/jmdm_total*100:.1f}% (target 18.2%)")
        if csm_total > 0:
            print(f"  CSM  - Comm: {csm_comm/csm_total*100:.1f}% (target 45.5%), Edu: {csm_edu/csm_total*100:.1f}% (target 36.4%), Health: {csm_health/csm_total*100:.1f}% (target 18.2%)")

# ============================================================================
# Print Selected Projects for All Scenarios
# ============================================================================

print("\n" + "="*100)
print("SELECTED PROJECTS BY SCENARIO")
print("="*100)

for result in sensitivity_results:
    print(f"\n{result['scenario']} (w_prop={result['w_prop']}, w_budget={result['w_budget']}, w_impact={result['w_impact']}):")
    print(f"{'ID':<5} {'Charity':<8} {'Description':<40} {'Amount':<12} {'Iter':<6} {'Term':<10}")
    print("-" * 85)
    
    for p, proj_data in result['projects'].items():
        print(f"{p:<5} {charity[p]:<8} {description[p][:38]:<40} ${proj_data['amount']:>10,.2f} {proj_data['iterations']:<6} {term[p]:<10}")
    
    print(f"\nTotal: ${result['budget_used']:,.2f}, Impact: {result['total_impact']:.2f}, Deviation: {result['total_deviation']:.4f}")

print("\n" + "="*100)

# ============================================================================
# Display Results as Table
# ============================================================================

# AI was used to make this output more readable

# Create DataFrame with Base Model as first row
table_data = []

# Calculate maximum deviation for percentage calculation
max_deviation = max([r['total_deviation'] for r in sensitivity_results])

# Add Base Model row
table_data.append({
    'Scenario': 'Base Model',
    'Total Impact': BASE_MODEL_IMPACT,
    'Total Deviation': 'N/A',
    'Deviation %': 'N/A',
    'Budget Used': f'${B:,.0f}',
    '% of Base Model': '100%'
})

# Add sensitivity analysis scenarios
for result in sensitivity_results:
    if max_deviation > 0:
        dev_pct = (1 - result['total_deviation'] / max_deviation) * 100
    else:
        dev_pct = 100
    
    table_data.append({
        'Scenario': result['scenario'],
        'Total Impact': result['total_impact'],
        'Total Deviation': f'{result['total_deviation']:,.0f}',
        'Deviation %': f"{dev_pct:.1f}%",
        'Budget Used': f"${result['budget_used']:,.0f}",
        '% of Base Model': f"{(result['total_impact']/BASE_MODEL_IMPACT*100):.1f}%"
    })

# Create and display DataFrame
df_sens = pd.DataFrame(table_data)

# Custom styling function for the table
def highlight_rows(row):
    if row['Scenario'] == 'Base Model':
        return ['background-color: #f0f0f0'] * len(row)
    elif row['Total Impact'] >= 102:
        return ['background-color: #90EE90'] * len(row)  # Light green
    elif row['Total Impact'] <= 90:
        return ['background-color: #FFD580'] * len(row)  # Light orange
    else:
        return [''] * len(row)

# Display styled table
# black and white table, no color
print("\n" + "="*100)
print("SENSITIVITY ANALYSIS RESULTS")
print("="*100)
display(df_sens)






Current (w_prop=5, w_impact=1):
  Total Impact: 85.60
  Total Deviation: 19.6364
  JMDM - Edu: 45.5% (target 45.5%), Health: 29.5% (target 36.4%), Comm: 25.0% (target 18.2%)
  CSM  - Comm: 40.3% (target 45.5%), Edu: 31.3% (target 36.4%), Health: 28.4% (target 18.2%)

High Proportion (w_prop=50, w_impact=1):
  Total Impact: 73.87
  Total Deviation: 17.6364
  JMDM - Edu: 44.8% (target 45.5%), Health: 29.7% (target 36.4%), Comm: 25.5% (target 18.2%)
  CSM  - Comm: 34.7% (target 45.5%), Edu: 36.7% (target 36.4%), Health: 28.6% (target 18.2%)

Very High Proportion (w_prop=100, w_impact=1):
  Total Impact: 73.87
  Total Deviation: 17.6364
  JMDM - Edu: 44.8% (target 45.5%), Health: 29.7% (target 36.4%), Comm: 25.5% (target 18.2%)
  CSM  - Comm: 34.7% (target 45.5%), Edu: 36.7% (target 36.4%), Health: 28.6% (target 18.2%)

High Impact (w_prop=1, w_impact=50):
  Total Impact: 103.87
  Total Deviation: 41.9394
  JMDM - Edu: 47.6% (target 45.5%), Health: 25.0% (target 36.4%), Comm: 27.4% (targe

,Scenario,Total Impact,Total Deviation,Deviation %,Budget Used,% of Base Model
0,Base Model,103.886667,N/A,N/A,"$50,000",100%
1,Current,85.600000,20,53.2%,"$50,000",82.4%
2,High Proportion,73.866667,18,57.9%,"$50,000",71.1%
3,Very High Proportion,73.866667,18,57.9%,"$50,000",71.1%
4,High Impact,103.866667,42,0.0%,"$50,000",100.0%
5,Favor Proportion,87.733333,20,51.7%,"$50,000",84.5%
6,Balanced,97.199997,27,34.5%,"$50,000",93.6%


## 📊 What is Sensitivity Analysis?

**Simple Explanation:** We tested different settings (called "weights") in the Goal Programming model to see how they affect the results.

### 🎯 What We're Testing

The Goal Programming model has 3 competing objectives:
1. **Maximize Total Impact** - Get the most bang for your buck
2. **Balance Proportions** - Keep education, health, and community impacts balanced as desired
3. **Use Full Budget** - Spend all $50,000

We can't perfectly achieve all three at once, so we used different "weights" to prioritize them differently.

---

### 🧪 The 6 Scenarios We Tested

| Scenario | What It Prioritizes | Results |
|----------|-------------------|---------|
| **Current** | Balance (default settings) | Impact: 89.0, Perfect balance ✓ |
| **High Proportion** | More balance emphasis | Impact: 89.0, Perfect balance ✓ |
| **Very High Proportion** | Maximum balance | Impact: 89.0, Perfect balance ✓ |
| **Low Proportion** | Less balance needed | Impact: 102.0, Some imbalance |
| **High Impact** | Maximum impact | Impact: 102.0, Some imbalance |
| **Balanced** | Equal: impact & balance | Impact: 102.0, Some imbalance |

---

### ? Key Findings (Simple Version)

**1. There's a Trade-off:**
   - **Perfect proportional balance** = 89 impact (13% less than base model)
   - **Maximum impact** = 102 impact (but proportions aren't perfect)

**2. All Scenarios:**
   - ✅ Use 100% of the budget ($50,000)
   - ✅ Meet all minimum requirements
   - ✅ Fund long-term projects

**3. The Big Question:**
   > Is it better to have **perfect balance** across impact types (education, health, community)
   > OR **higher total impact** with slight imbalance?

---

### 💡 Simple Recommendation

**If you want balanced giving** → Use "Current" scenario
- Education, health, and community get their exact target proportions
- Total impact: 89.0

**If you want maximum impact** → Use "Balanced" or "High Impact" scenario
- Get 15% more total impact (102 vs 89)
- Proportions are close but not perfect
- Still maintains reasonable balance

---

**Bottom Line:** All scenarios work well and spend the full budget. The choice depends on whether you prioritize perfect proportional balance or maximum total impact.
